In [1]:
import torch
from cac import test_accuracy
from hash_model import HASH_Net
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device("cuda")
hash_bits = 64 # 假设哈希编码的位数
model_name = "vgg11"
batch_size = 10
noise_rate=0.4

with open('../labels/64_cifar10_10_class.pkl', 'rb') as f:
    label_hash_codes= torch.load(f)
label_hash_codes.to(device)

# 1. 加载CIFAR-10数据集
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)


# 确保你的模型架构定义与保存前一致
model = HASH_Net(model_name,hash_bits).to(device)  # YourModelArchitecture是你的模型架构
model.load_state_dict(torch.load(f'./model/nr_{noise_rate}_model.pth'))  # 加载保存的模型参数
model.eval()  # 设置为评估模式
accuracy = test_accuracy(model, testloader, label_hash_codes,device)
print(f'Test Accuracy: {accuracy}%')

/root/anaconda3/envs/deephashh/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/root/anaconda3/envs/deephashh/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Accuracy: 92.61%


In [2]:
import torch
torch.cuda.device_count()

1